In [1]:
import syft as sy
import torch
from torch import nn, optim

In [2]:
hook = sy.TorchHook(torch)

In [3]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
central_server = sy.VirtualWorker(hook, id='central_server')

In [31]:
bob.add_workers([alice, central_server])
alice.add_workers([bob, central_server])
central_server.add_workers([alice, bob])

<VirtualWorker id:central_server #tensors:0>

In [5]:
raw_data = torch.tensor([[0., 0], [0, 1], [1, 0], [1, 1]], requires_grad=True)
target = torch.tensor([[0.],[1], [1], [1]], requires_grad=True)

In [6]:
data_bob = raw_data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [7]:
bob._objects

{27214138956: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 61579309777: tensor([[0.],
         [1.]], requires_grad=True)}

In [8]:
central_server._objects

{}

In [9]:
data_alice = raw_data[2:].send(alice)
target_alice = target[2:].send(alice)

In [25]:
alice._objects

{85241161753: tensor([[1., 0.],
         [1., 1.]], requires_grad=True), 29055826749: tensor([[1.],
         [1.]], requires_grad=True), 24208730054: Parameter containing:
 tensor([[0.0905, 0.6762]], requires_grad=True), 41820619397: Parameter containing:
 tensor([0.4906], requires_grad=True)}

In [20]:
central_server.id

'central_server'

In [27]:
model = nn.Linear(2, 1)
model

Linear(in_features=2, out_features=1, bias=True)

In [13]:
dataset = [(data_bob, target_bob), (data_alice, target_alice)]

In [26]:
bob_model = model.copy().send(bob)
alice_model = model.copy().send(alice)

Linear(in_features=2, out_features=1, bias=True)

In [28]:
bob_optimizer = optim.SGD(bob_model.parameters(), lr=0.001)
alice_optimizer = optim.SGD(alice_model.parameters(), lr=0.001)
bob_optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [16]:
bob_criterion = nn.MSELoss()
alice_criterion = nn.MSELoss()

In [22]:
bob_pred = bob_model(data_bob)

In [23]:
bob_pred

(Wrapper)>[PointerTensor | me:12037785948 -> bob:2060883996]

In [33]:
bob_pred.get()

tensor([[0.4906],
        [1.1669]], requires_grad=True)

In [30]:
central_server

'central_server'

In [17]:
def train(iterations=20):
    
    for i in range(iterations):

        for _data, _target in dataset:
            print(_target.id)

#             # Move the model to the various locations
#             model = model.send(_data.location)

#             # Remove the accumulating Gradients
#             optimizer.zero_grad()

#             # Make the prediction
#             pred = model(_data)

#             # Calculate the Loss
#             loss = ((pred- _target)**2).sum()

#             # Perform the back propagations
#             loss.backward()

#             # Update the weights
#             optimizer.step()

#             # Get Smarter model back
#             model = model.get()
#             print(loss.get())

In [18]:
train()

98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
98159025515
16281226221
